# Aufgabenstellung

Für ein Heizsystem wird ein Glastubus verwendet. Durch eine regelbare Stromzufuhr wird Wärme generiert.
Je mehr Strom eingespeist wird, desto wärmer wird der Tubus.
Dieser Tubus gibt Wärme an seine Umgebung ab.
Dies geschieht Proportional zu der Oberfläche des Tubus
und des Temperaturunterschiedes zwischen dem Tubus und der Umgebungstemperatur.
Das Gleichgewicht des Temperatur-Verlustes und der Wärmeerzeugung lässt sich durch folgende Differentialgleichung ausdrücken:

<img src="dgl.png" />

Das Ziel ist es, die Temperatur möglichst schnell mit wenig Schwankungen auf ein gewünschtes Niveau zu bringen.


# Aufgabe 1
In dieser Aufgabe soll die Differentialgleichung an einen Satz von Messdaten "gefittet" werden.
Hiebei sollen $m$ (Masse), sowie $A_s$ (Oberfläche), ermittelt werden.
Die Datei enthält Ergebnisse des Systems auf eine Sprungantwort.

In [2]:
%matplotlib inline
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt


In [3]:
my_data = np.genfromtxt('measurement_data.csv', delimiter=';')
xdata = my_data[:,0]
ydata = my_data[:,1]

OSError: measurement_data.csv not found.

In [ ]:
#def func(t, q, m):
#  return q*t/(m*1078)
def func(t, q, As, Ta, m):
    hs = 5
    return ((-(q/(As*hs)+Ta)+Ta)*np.e**(-(As*hs*t)/(1078*m))+q/(As*hs)+Ta)

In [ ]:
popt, pcov = curve_fit(func, xdata, ydata, p0=(1000,1,20, 15), bounds=([0, 0.1, -65, 10],[2000, 2, 300, 20]))

In [ ]:
q  = popt[0]
As = popt[1]
Ta = popt[2]
m = popt[3]

residuals = ydata - func(xdata,q, As, Ta, m)
fres = sum(np.abs(residuals))
print('fres: %6.2f ' % fres)
print('q: %6.2f' % q)
print('As: %6.2f' % As)
print('Ta: %6.2f' % Ta)
print('m: %6.2f' % m)

#print('Ta: %6.2f' % Ta)

In [ ]:
curvex= np.linspace(0,20000,10000)
curvey= func(curvex,q,As,Ta, m)

In [ ]:
plt.plot(xdata,ydata,'*')
plt.plot(curvex,curvey,'r')
plt.xlabel("X-data")
plt.ylabel("Y-data")

# Teil 2
ab jetzt werden die berechneten Werte für m und As eingesetzt, Ta und nocheinmal gefittes anhand der festen Werte

In [ ]:
m = 16.49331431
As = 1.256637061
hs = 5

In [ ]:
def func(t, q, Ta):
    return (-((q/(As*hs)+Ta)+Ta)*np.e**(-(As*hs*t)/(1078*m))+q/(As*hs)+Ta)

In [ ]:
popt, pcov = curve_fit(func, xdata, ydata, p0=(1000, 30), bounds=([0, -20],[2000, 100]))
q  = popt[0]
Ta = popt[1]

residuals = ydata - func(xdata ,q, Ta)
fres = sum(np.abs(residuals))
print('fres: %6.2f ' % fres)
print('q: %6.2f' % q)
print('Ta: %6.2f' % Ta)

In [ ]:
curvex= np.linspace(0,20000,10000)
curvey= func(curvex, q, Ta)
plt.plot(xdata,ydata,'*')
plt.plot(curvex,curvey,'r')
plt.xlabel("X-data")
plt.ylabel("Y-data")

# -> wähle Ta als 14 Kelvin/Grad C
Nun ermittle maximale Temperatur des Tubus bei maximaler Heizleistung von 2kW

Berechnung: q_in = q_out, also Energieverlust pro Zeit ist gleich der Heizleistung von 2kW

In [ ]:
Ta = 7.11

In [ ]:
#2 - hs*As*(T-Ta)= 0
T_max = 2000/(hs*As)+Ta 
print ("Die maximal zu erreichende Temperatur liegt bei: {} (bei festem Ta von {}.".format(T_max, Ta))

def maxT(Ta_local):
    return 2000/(hs*As)+Ta_local

aussentemperatur = np.linspace(-20,100,120)
T_max= maxT(aussentemperatur)
plt.plot(aussentemperatur,aussentemperatur,'r')
plt.xlabel("Ta")
plt.ylabel("T_max")

2000/(hs*1.36)+20

Das System ist in dem Sinne linear, dass bei Doppelter Leistung sich auch eine doppelt so hohe Temperatur einstellt.

In [ ]:
curvex= np.linspace(0,20000,1000)
curvey= func(curvex, 1, Ta)
plt.plot(curvex,curvey,'r')
plt.xlabel("X-data")
plt.ylabel("Y-data")

Das die Messadten im Negativen Bereich beginnen, sieht man in dem hier präsentierten Graphen. Die Temperatur steigt nimmt in den ersten 5000 Sekunden um 12 Grad zu, jedoch stellt sich die Endtemperatur erst nach ca 13000 Sekunden, bei 0 Grad ein. 

In [ ]:
f = 1000000
#IST DAS SCHUMMELEI? MÖGLICH...
def u(t, f):
    return 1000*np.sin(np.pi*((f*2*t)%2))

In [ ]:
for t in range(10):
    print(t, u(t/2, f))
#stelle diese Aufgabe zurück, frage mal nach: ist immer 0 gewollt? was für ein Offset?

In [ ]:
# simuliere constante von 1kW -> berechne erzielte Temperatur bei 1kW leistung
p_max = 1000
T_1kW = p_max/(hs*As)+Ta
print(T_1kW)

# Funktion der Temperatur, welche bei der einer Temperatur anfängt, welche bei einer Leistung von 1kW entsteht
def func_with_pre_temp(t, q, T_pre):
    return (-(q/(As*hs)+Ta)*np.e**(-(As*hs*t)/(1078*m))+q/(As*hs)+Ta)+T_pre

curvex= np.linspace(0,20000,10000)
curvey= func_with_pre_temp(curvex, q, T_1kW)
plt.plot(curvex,curvey,'r')
plt.xlabel("X-data")
plt.ylabel("Y-data")

In [ ]:
def func_1(t, q, start_temp):
    return (-((q/(As*hs)+Ta)-start_temp)*np.e**(-(As*hs*t)/(1078*m))+q/(As*hs)+Ta)
    
q = 1000
time_offset = 0
start_temp = 0
curvex = np.arange(0, 300*60, 0.5)
curvey = np.arange(0, 300*60, 0.5)


m = 16.49
As = 1.35
Ta = 7.11

for i in range(len(curvex)):
    if(curvex[i] == 1800):
        q = 2000
        time_offset = 1800
        start_temp = curvey[i-1]
    curvey[i] = func_1(curvex[i]-time_offset, q, start_temp)

In [ ]:
plt.plot(curvex, curvey, 'r')

In [ ]:
print(curvey[4000])

In [ ]:
curvex_3 = np.arange(0, 250*60, 0.5)
curvey_3 = np.arange(0, 250*60, 0.5)

start_temp = 0
for i in range(len(curvex_3)):
    q = 1000+(np.random.rand(1)-0.5)*1000
    curvey_3[i] = func_1(0.5, q, start_temp)
    start_temp =  curvey_3[i]

In [ ]:
plt.plot(curvex_3, curvey_3, 'r')